# cytoscape_node_to_shapely_node
Convert any cytoscape node shape to a shapely shape

## Methods:
### cyto_node_to_shapely_node(shape, original_height, original_width, x_location, y_location, buffer)
Converts a cytoscape node to a shapely representation of the node
#### Parameters:
* shape: shape of cytoscape node (either 'TRIANGLE', 'RECTANGLE', 'ROUND_RECTANGLE', 'PARALLELOGRAM', 'DIAMOND', 'VEE', 'HEXAGON', 'OCTAGON', or 'ELLIPSE')
* original_height: the original height of the cytoscape node
* original_width: the original width of the cytoscape node
* x_location: the original x coordinate of the cytoscape node
* y_location: the original y coordinate of the cytoscape node
* buffer: the difference between the size of the shapely shape being created and the original size of the cytoscape node. A positive buffer will make bigger shapely shapes.

#### Returns:
* shape: shapely shape

In [1]:
import math
from math import sqrt

import shapely
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import box

#Methods for making shapes
def cyto_node_to_shapely_node(shape, original_height, original_width, x_location, y_location, buffer):
    width = original_width + buffer
    height = -1 * (original_height + buffer)
    if shape.upper() == 'TRIANGLE':
        return _cyto_to_shapely_triangle(width, height, x_location, y_location)
    elif shape.upper() == 'RECTANGLE' or shape.upper() == 'ROUND_RECTANGLE':
        return _cyto_to_shapely_rectangle(width, height, x_location, y_location)
    elif shape.upper() == 'PARALLELOGRAM':
        return _cyto_to_shapely_parallelogram(width, height, x_location, y_location)
    elif shape.upper() == 'DIAMOND':
        return _cyto_to_shapely_diamond(width, height, x_location, y_location)
    elif shape.upper() == 'VEE':
        return _cyto_to_shapely_vee(width, height, x_location, y_location)
    elif shape.upper() == 'HEXAGON':
        return _cyto_to_shapely_hexagon(width, height, x_location, y_location)
    elif shape.upper() == 'OCTAGON':
        return _cyto_to_shapely_octagon(width, height, x_location, y_location)
    elif shape.upper() == 'ELLIPSE':
        return _cyto_to_shapely_ellipse(width, height, x_location, y_location)

def _cyto_to_shapely_triangle(width, height, x_location, y_location):
    top = (x_location, y_location + (height / 2))
    left = (x_location - (width / 2), y_location - (height / 2))
    right = (x_location + (width / 2), y_location - (height / 2))
    return Polygon([left, top, right])

def _cyto_to_shapely_rectangle(width, height, x_location, y_location):
    minx = x_location - (width / 2)
    miny = y_location - (height / 2)
    maxx = x_location + (width / 2)
    maxy = y_location + (height / 2)
    return box(minx, miny, maxx, maxy)

def _cyto_to_shapely_parallelogram(width, height, x_location, y_location):
    offset = width / 3
    return Polygon([
        (x_location - (width / 2), y_location + (height / 2)),
        (x_location + (width / 2) - offset, y_location + (height / 2)),
        (x_location + (width / 2), y_location - (height / 2)),
        (x_location - (width / 2) + offset, y_location - (height / 2))
    ])

def _cyto_to_shapely_diamond(width, height, x_location, y_location):
    right = (x_location + (width / 2), y_location)
    left = (x_location - (width / 2), y_location)
    top = (x_location, y_location + (height / 2))
    bottom = (x_location, y_location - (height / 2))
    return Polygon([right, top, left, bottom])

def _cyto_to_shapely_vee(width, height, x_location, y_location):
    left = (x_location - (width / 2), y_location + (height / 2))
    middle = (x_location, y_location + (height / 6))
    right = (x_location + (width / 2), y_location + (height / 2))
    bottom = (x_location, y_location - (height / 2))
    return Polygon([left, middle, right, bottom])

def _cyto_to_shapely_hexagon(width, height, x_location, y_location):
    radius = 2
    short_leg = 1
    long_leg = sqrt(3)
    hexagon = Polygon([
        (x_location - radius, y_location), 
        (x_location - short_leg, y_location + long_leg), 
        (x_location + short_leg, y_location + long_leg), 
        (x_location + radius, y_location), 
        (x_location + short_leg, y_location - long_leg), 
        (x_location - short_leg, y_location - long_leg)
    ])
    return shapely.affinity.scale(hexagon, xfact = width / 4, yfact = height / 4)#(2 * math.sqrt(3)))

def _cyto_to_shapely_octagon(width, height, x_location, y_location):
    short_offset = 1
    long_offset = 1 + sqrt(2)
    octagon = Polygon([
        (x_location - long_offset, y_location - short_offset),
        (x_location - long_offset, y_location + short_offset),
        (x_location - short_offset, y_location + long_offset),
        (x_location + short_offset, y_location + long_offset),
        (x_location + long_offset, y_location + short_offset),
        (x_location + long_offset, y_location - short_offset),
        (x_location + short_offset, y_location - long_offset),
        (x_location - short_offset, y_location - long_offset)
    ])
    diameter = 2 * sqrt(4 + 2 * sqrt(2))
    return shapely.affinity.scale(octagon, xfact = width / diameter, yfact = height / diameter)

def _cyto_to_shapely_ellipse(width, height, x_location, y_location):
    circle = Point(x_location, y_location).buffer(1)
    return shapely.affinity.scale(circle, xfact = (width / 2), yfact = (height / 2))